# Домашнее задание

### Д/з из четырех пунктов:
* Улучшение `fit_generator`
* Сравнение двух ReLU (разные активации)
* Испорченный батч-норм 
* "Сырые" данные. 

### Что нужно сделать
* Следовать инструкциям в каждом из пунктов.
* Результатами вашей работы будет ноутбук с доработанным кодом + архив с директорией с логами `tensorboard` `logs/`, в который вы запишите результаты экспериментов. Подробности в инструкциях ниже.
* Можно и нужно пользоваться кодом из файла `utils`, **но** весь код модифицируйте, пожалуйста, в ноутбуках! Так мне будет проще проверять.

**Загрузка tensorboard в ноутбук**

Можете попробовать использовать его так на свой страх и риск :)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

**Импорты**

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from typing import Callable

### Импорт слоев для д/з

In [2]:
from utils import BatchNormFlawed, Dense, DenseSmart, Sequential, MNISTSequence

### Загрузка данных

> Здесь ничего менять не нужно

In [3]:
(X_tr, y_tr), (X_test, y_test) = keras.datasets.mnist.load_data()

In [4]:
train_seq = MNISTSequence(X_tr, y_tr, 128)
test_seq = MNISTSequence(X_test, y_test, 128)

**Очистка данных**

In [ ]:
!rm -rf logs/*

In [151]:
!mkdir logs

In [5]:
keras.backend.clear_session()

## 1. Улучшение fit_generator

Улучшите метод `fit_generator` так, чтобы он:
* Записывал значения градиентов для всех переменных при помощи `tf.summary.histogram` 
* Записывал значения ошибки и метрики на валидации с помощью `tf.summary.scalar`

Затем сделайте monkey patch класса sequential обновленным методом (следующая ячейка за методом `fit_generator`).

In [6]:
def fit_generator(self, train_seq, eval_seq, epoch, loss, optimizer, writer=None):
    history = dict(train=list(), val=list())

    train_loss_results = list()
    val_loss_results = list()

    train_accuracy_results = list()
    val_accuracy_results = list()

    step = 0
    for e in range(epoch):
        p = tf.keras.metrics.Mean()
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_loss_avg_val = tf.keras.metrics.Mean()

        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        epoch_accuracy_val = tf.keras.metrics.SparseCategoricalAccuracy()

        for x, y in train_seq:
            with tf.GradientTape() as tape:
                """
                Обратите внимание! Если записывать гистограмму каждый шаг,
                обучение будет идти очень медленно. Поэтому записываем данные 
                каждый i-й шаг.
                """
                if step % 50 == 0:
                    prediction = self._forward(x, writer, step)
                else:
                    prediction = self._forward(x)
                loss_value = loss(y, prediction)
                     
            ###############################################################
            #                                                             #
            # Добавьте запись градиентов в гистограммы                    #
            #                                                             #
            ###############################################################
        
            gradients = tape.gradient(loss_value, self._trainable_variables)
            
            if step % 50 == 0:
                """
                Пример того, как можно дать всем градиентам уникальные имена. 
                Обратите внимание! Создание grad_names лучше вынести из цикла,
                чтобы не пересоздавать список на каждом шаге! 
                """
                grad_names = list()
                for layer in self._layers:
                    for var_num, var in enumerate(layer.get_trainable()):
                        grad_names.append(f"grad_{layer.name}_{var_num}")

                if writer is not None:
                    with writer.as_default():
                        for i, gradient in enumerate(gradients):
                            tf.summary.histogram(grad_names[i], gradients[i], step=step)
            
            optimizer.apply_gradients(zip(gradients, self._trainable_variables))
            epoch_accuracy.update_state(y, prediction)
            epoch_loss_avg.update_state(loss_value)

            if step % 50 == 0:
                with writer.as_default():
                    tf.summary.scalar('train_accuracy', epoch_accuracy.result().numpy(), step=step)
                    tf.summary.scalar('train_loss', epoch_loss_avg.result().numpy(), step=step)

            step += 1

        train_accuracy_results.append(epoch_accuracy.result().numpy())
        train_loss_results.append(epoch_loss_avg.result().numpy())

        for valbatch_n, (x, y) in enumerate(eval_seq):
            prediction = self._forward(x)
            loss_value = loss(y, prediction)
            epoch_loss_avg_val.update_state(loss_value)
            epoch_accuracy_val.update_state(y, prediction)
     
            ###############################################################
            #                                                             #
            # Добавьте сохранение метрики и функции ошибки на валидации   #
            #                                                             #
            ###############################################################
            
        # Не вижу смысла считать качество валидации на каких-то отдельных батчах, 
        # запишем качество валидации на всей тестовой выборке в конце каждой эпохи
        if writer is not None:
            with writer.as_default():
                tf.summary.scalar('val_accuracy', epoch_accuracy_val.result().numpy(), step=e*len(train_seq))
                tf.summary.scalar('val_loss', epoch_loss_avg_val.result().numpy(), step=e*len(train_seq))
            
        val_accuracy_results.append(epoch_accuracy_val.result().numpy())
        val_loss_results.append(epoch_loss_avg_val.result().numpy())

        print("Epoch {}: Train loss: {:.3f} Train Accuracy: {:.3f}".format(e + 1,
                                                                           train_loss_results[-1],
                                                                           train_accuracy_results[-1]))
        print("Epoch {}: Val loss: {:.3f} Val Accuracy: {:.3f}".format(e + 1,
                                                                       val_loss_results[-1],
                                                                       val_accuracy_results[-1]))
        print('*' * 20)

    return None

In [7]:
# Monkey patch: обновляем метод
Sequential.fit_generator = fit_generator

## 2. Сравнение двух ReLU (разные активации)

Запустите два эксперимента ниже. Сравните результаты - значения метрик после каждого из них.

Запустите tensorboard, изучите распределения активаций, градиентов и т.д. для `relu` и `smart_dense_relu`. 

Подумайте, почему в одном случае сеть обучается плохо, а в другом - хорошо. Вставьте в ноутбук (или напишите список названий) тех графики из tensorboard, которые, по вашему мнению, это иллюстрируют, и напишите, почему.


Команда для запуска tensorboard в bash:

`$ tensorboard --logdir logs/`

**Ваш комментарий:**

<span style="color:red ">__На мой взгляд, больше всего информации дают графики с градиентами.__</span>

<img src="img/relu_vs_smart_dense_relu.png" style="width:70%">

<span style="color:red ">__У слоёв с правильной инициализацией мы видим огромный градиент в самом начале, чего не наблюдается у слоя с обчной инициализацией. Это означает, что начальное положение весов находится где-то на грани "обрыва", и алгоритм быстро находит хороший минимум.__</span>

<span style="color:red ">__В дальшейшем градиенты у слоя с правильной инициализацией в среднем гораздо меньше, чем у слоя с обычной инициализацией. Неудивительно, ведь мы уже в хорошем минимуме.__</span>

<span style="color:red ">__Однако, не все градиенты ведут себя подобным образом. Исключение по непонятной мне причине составляет bias второго полносвязного слоя (внимание на масштаб). В то время, как градиенты слоя с обыной инициализацией как правило не превышают 2, градиенты слоя с правильной инициализацией в несколько раз больше.__</span>

<img src="img/relu_vs_smart_dense_relu_2.png" style="width:70%">

In [8]:
writer = tf.summary.create_file_writer("logs/relu")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 100, tf.nn.relu, 'dense2'), 
                   Dense(100, 10, tf.nn.softmax, 'output'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 11.731 Train Accuracy: 0.269
Epoch 1: Val loss: 9.938 Val Accuracy: 0.379
********************
Epoch 2: Train loss: 8.771 Train Accuracy: 0.452
Epoch 2: Val loss: 7.943 Val Accuracy: 0.504
********************
Epoch 3: Train loss: 7.674 Train Accuracy: 0.521
Epoch 3: Val loss: 7.131 Val Accuracy: 0.555
********************
Epoch 4: Train loss: 7.089 Train Accuracy: 0.557
Epoch 4: Val loss: 6.743 Val Accuracy: 0.579
********************
Epoch 5: Train loss: 6.719 Train Accuracy: 0.580
Epoch 5: Val loss: 6.434 Val Accuracy: 0.598
********************
Epoch 6: Train loss: 6.470 Train Accuracy: 0.596
Epoch 6: Val loss: 6.282 Val Accuracy: 0.608
********************
Epoch 7: Train loss: 6.311 Train Accuracy: 0.607
Epoch 7: Val loss: 6.152 Val Accuracy: 0.616
********************
Epoch 8: Train loss: 6.203 Train Accuracy: 0.613
Epoch 8: Val loss: 6.063 Val Accuracy: 0.622
********************
Epoch 9: Train loss: 6.121 Train Accuracy: 0.618
Epoch 9: Val loss: 6.022 Val A

In [9]:
writer = tf.summary.create_file_writer("logs/relu_smart_dense")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense2'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'output'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 0.323 Train Accuracy: 0.904
Epoch 1: Val loss: 0.183 Val Accuracy: 0.945
********************
Epoch 2: Train loss: 0.129 Train Accuracy: 0.962
Epoch 2: Val loss: 0.136 Val Accuracy: 0.960
********************
Epoch 3: Train loss: 0.086 Train Accuracy: 0.975
Epoch 3: Val loss: 0.122 Val Accuracy: 0.964
********************
Epoch 4: Train loss: 0.064 Train Accuracy: 0.981
Epoch 4: Val loss: 0.110 Val Accuracy: 0.969
********************
Epoch 5: Train loss: 0.047 Train Accuracy: 0.985
Epoch 5: Val loss: 0.133 Val Accuracy: 0.962
********************
Epoch 6: Train loss: 0.038 Train Accuracy: 0.988
Epoch 6: Val loss: 0.148 Val Accuracy: 0.958
********************
Epoch 7: Train loss: 0.031 Train Accuracy: 0.991
Epoch 7: Val loss: 0.144 Val Accuracy: 0.961
********************
Epoch 8: Train loss: 0.028 Train Accuracy: 0.991
Epoch 8: Val loss: 0.125 Val Accuracy: 0.967
********************
Epoch 9: Train loss: 0.025 Train Accuracy: 0.992
Epoch 9: Val loss: 0.134 Val Ac

## 3.a Испорченный батч-норм 

Запустите два эксперимент ниже. 

Почему обучение не идет? В чем ошибка в слое `BatchNorm`? Изучите и исправьте код метода `__call__` (Шаблон находится ниже под блоком с экспериментом.).

Можно пользоваться tensorboard, если он нужен.

## ReLU + Batch Norm

In [10]:
writer = tf.summary.create_file_writer("logs/relu_bn")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFlawed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 2.528 Train Accuracy: 0.111
Epoch 1: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 2: Train loss: 2.301 Train Accuracy: 0.112
Epoch 2: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 3: Train loss: 2.301 Train Accuracy: 0.112
Epoch 3: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 4: Train loss: 2.301 Train Accuracy: 0.112
Epoch 4: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 5: Train loss: 2.301 Train Accuracy: 0.112
Epoch 5: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 6: Train loss: 2.301 Train Accuracy: 0.112
Epoch 6: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 7: Train loss: 2.301 Train Accuracy: 0.112
Epoch 7: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 8: Train loss: 2.301 Train Accuracy: 0.112
Epoch 8: Val loss: 2.301 Val Accuracy: 0.113
********************
Epoch 9: Train loss: 2.301 Train Accuracy: 0.112
Epoch 9: Val loss: 2.301 Val Ac

**Класс, который нужно исправить**

In [11]:
class BatchNormFixed(BatchNormFlawed):
    def __call__(self, x, writer=None, step=None):
        """
        Исправьте блок кода ниже так, чтобы модель обучалась, не появлялись значения loss = NaN        """
        mu = tf.reduce_mean(x, axis=0)
        sigma = tf.dtypes.cast(tf.math.reduce_std(x, axis=0), tf.double)
        normed = (x - mu) / tf.add(sigma, tf.constant(0.001, dtype=tf.double))
        out = normed * self._gamma + self._beta
        """
        Конец блока, который нужно исправить
        """
        
        if writer is not None:
            with writer.as_default():
                tf.summary.histogram(self.name + '_beta', self._beta, step=step)
                tf.summary.histogram(self.name + '_gamma', self._gamma, step=step)
                tf.summary.histogram(self.name + '_normed', normed, step=step)
                tf.summary.histogram(self.name + '_out', out, step=step)
                tf.summary.histogram(self.name + '_sigma', sigma, step=step)
                tf.summary.histogram(self.name + '_mu', mu, step=step)
        return out

## 3.b Исправленный батч-норм 

Запустите эксперимент ниже. 

Обучается ли сеть? Идет ли процесс обучения лучше, чем в эксперименте с ReLU? 

Сравните обучение сетей c ReLU и слоем `Dense` (а не `DenseSmart`!) и ReLU с BatchNorm в tensorboard, как в задании 2.
Напишите ваши выводы.

_Обратите внимание, что слева в интерфейсе tensorboard есть меню, которое позволяет выключать визуализацию ненужных экспериментов._

**Ваш комментарий:**

<span style="color:red ">__Судя по всему, проблема была в том, что sigma становится близкой к 0, или 0, из за этого уходим с ошибкой деления на ноль. Странно только, что не вываливается ексепшен.__</span>

<img src="img/broken_batchnorm.png" style="width:70%">

<span style="color:red ">__Сесть с BatchNorm обучается лучше, чем без него. Видимо, он помогает)__</span>

In [12]:
writer = tf.summary.create_file_writer("logs/relu_bn_fixed")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFixed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 8.798 Train Accuracy: 0.428
Epoch 1: Val loss: 5.953 Val Accuracy: 0.595
********************
Epoch 2: Train loss: 2.866 Train Accuracy: 0.668
Epoch 2: Val loss: 0.695 Val Accuracy: 0.784
********************
Epoch 3: Train loss: 0.597 Train Accuracy: 0.816
Epoch 3: Val loss: 0.504 Val Accuracy: 0.845
********************
Epoch 4: Train loss: 0.467 Train Accuracy: 0.858
Epoch 4: Val loss: 0.419 Val Accuracy: 0.875
********************
Epoch 5: Train loss: 0.395 Train Accuracy: 0.880
Epoch 5: Val loss: 0.367 Val Accuracy: 0.892
********************
Epoch 6: Train loss: 0.345 Train Accuracy: 0.896
Epoch 6: Val loss: 0.329 Val Accuracy: 0.902
********************
Epoch 7: Train loss: 0.307 Train Accuracy: 0.907
Epoch 7: Val loss: 0.298 Val Accuracy: 0.913
********************
Epoch 8: Train loss: 0.275 Train Accuracy: 0.917
Epoch 8: Val loss: 0.273 Val Accuracy: 0.920
********************
Epoch 9: Train loss: 0.246 Train Accuracy: 0.926
Epoch 9: Val loss: 0.251 Val Ac

## 4. "Сырые" данные. 

Что будет, если заставить сеть обучаться на сырых данных? 

Напишите такую функцию `preprocess`, которая не делает min-max scaling изображений и оставляет их в изначальном диапазоне. Не убирайте reshape! Конечно, она должна менять форму матрицы входных данных от `(n x 28 x 28)` к `(n x 784)`. 

Затем передайте функцию в MNISTSequence, создайте новую train- и test- последовательности запустите эксперимент, используя их как входные данные. 

Сравните результаты экспериментов c `DenseSmart` + ReLU и обработанными изображениями и `DenseSmart` + ReLU c необработанными изображениями. 

Обучается ли нейросеть? Если нет, то почему? Сделайте выводы, как в задании 2.

**Ваш комментарий:**

<span style="color:red ">__Нейронная сеть легко может проделать scaling самостоятельно путём установки нужных весов в первом слое. Однако, это потребует множества шагов в связи с тем, что веса нейронной сети инициализируются небольшими значениями, и шаг обучения тоже обычно весьма невилик, чтобы не перепрыгнуть удачный оптимум. По этой причине нейронная сетья будет хуже сходится в случае, если для исходных признаков не выполнен minmax scaling__</span>

<img src="img/val.png" style="width:70%">

**Шаблон Preprocess**

In [40]:
def preprocess(X, y):
    return X.reshape((-1, 28*28)), y

**Создание генераторов**

In [41]:
train_seq_raw = MNISTSequence(X_tr, y_tr, 128, preprocess=preprocess)
test_seq_raw = MNISTSequence(X_test, y_test, 128, preprocess=preprocess)

**Эксперимент**

In [43]:
writer = tf.summary.create_file_writer("logs/raw")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq_raw, test_seq_raw, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer
                          )

Epoch 1: Train loss: 13.083 Train Accuracy: 0.188
Epoch 1: Val loss: 12.774 Val Accuracy: 0.207
********************
Epoch 2: Train loss: 12.778 Train Accuracy: 0.207
Epoch 2: Val loss: 12.696 Val Accuracy: 0.212
********************
Epoch 3: Train loss: 12.511 Train Accuracy: 0.224
Epoch 3: Val loss: 12.651 Val Accuracy: 0.215
********************
Epoch 4: Train loss: 12.391 Train Accuracy: 0.231
Epoch 4: Val loss: 11.516 Val Accuracy: 0.285
********************
Epoch 5: Train loss: 11.795 Train Accuracy: 0.268
Epoch 5: Val loss: 11.426 Val Accuracy: 0.291
********************
Epoch 6: Train loss: 11.739 Train Accuracy: 0.272
Epoch 6: Val loss: 11.307 Val Accuracy: 0.299
********************
Epoch 7: Train loss: 11.421 Train Accuracy: 0.291
Epoch 7: Val loss: 11.463 Val Accuracy: 0.289
********************
Epoch 8: Train loss: 11.679 Train Accuracy: 0.275
Epoch 8: Val loss: 11.600 Val Accuracy: 0.280
********************
Epoch 9: Train loss: 11.668 Train Accuracy: 0.276
Epoch 9: Val l